1. 幂运算
* 一个正整数可以表示为若干指数不重复的2的次幂的和：
$$b=c_{k-1}2^{k-1}+c_{k-2}2^{k-2}+……+c_02^0$$
* 于是$a^b$可以表示为:
$$a^b=a^{c_{k-1}2^{k-1}}*a^{c_{k-2}2^{k-2}}*……*a^{c_02^0}$$
* 使用到快速幂运算的知识：    
(a + b) % p = (a % p + b % p) % p    
(a – b) % p = (a % p – b % p) % p    
(a * b) % p = (a % p * b % p) % p    
(a^b) % p = ((a % p)^b) % p    
* related blog
  * [算法学习笔记(4):快速幂](https://zhuanlan.zhihu.com/p/95902286)
  * [POJ 1995 Raising Modulo Numbers](http://www.hankcs.com/program/cpp/poj-1995-raising-modulo-numbers.html)

1.1 $a^b$

In [18]:
def recupow(a,b):
    if b == 0:
        return 1
    elif b %2 == 1:
        return a * recupow(a,b-1)
    else:
        temp = recupow(a,b/2)
        return temp * temp

def pow(a,b):
    ans = 1
    while b:
        if b &1:
            ans *= a
        a *= a
        b >>= 1
    return ans 


2


1.2 $a^b\%p,1 \leqslant a,b,p \leqslant 10^9$

In [ ]:
def recuqpow(a,b,q):
    if b == 0:
        return 1
    elif b%2 == 1:
        return a * recuqpow(a,b-1,q) % q
    else:
        temp = recuqpow(a,b/2)
        return temp * temp % q

def qpow(a,b,p):    # a^b%p
    ans = 1 % p
    while b:
        if b&1:
            ans = (ans * a) % p             # only when b&1(c_i) ==1, multiple current a to ans
        a = (a * a) % p                     # update each time a
        b = b>>1
    return ans
print(qpow(5,7,3))

1.3 $a*b\%p,1 \leqslant a,b,p \leqslant 10^18$

In [ ]:
def pmul(a,b,p):
    ans = 1 % p
    while b:
        if b&1:
            ans = (ans+a) % p
        a = a * 2 % p
    return ans

2. 二进制状态压缩
将一个长度为`m`的`bool`数组用一个`m`位二进制整数`n`中表示并存    

|操作|运算|
|:--:|:--:|
|取第k位|(n>>k)&1|
|取出第0~k-1位(后k位)|n&((1<<k)-1)|
|第k位取反|n^(1<<k)|
|第k位赋1|n\|(1<<k)|
|第k位赋0|n&(~(1<<k))|

* related blog       
[那么什么是状态压缩呢？为什么能用到求最短哈密顿路径呢](http://www.soolco.com/post/57252_1_1.html)    
**所以所谓的状态压缩就是把 一个状态集合用一个二进制数表示，我们只关心状态集合**

2.1 Hamilton 距离    
给定一张$n(n \leqslant 20)$个点的带权无向图，点从$0~n-1$标号，求起点0到终点n-1的最短Hamilton路径
* 状态转移公式
F[state,j] = min{F[state_k,k] + weight[k,j]}, state_k=state除掉j之后的集合,state_k要包含k
F[state,j] = min{F[state^(1<<j),k] + weight(k,j)}    
F[state,j]：“点被经过的状态”对应二进制为state，且目前处于点j时的最短路径

In [ ]:
F = [[float('inf')]*20 for _ in range(1<<20)]       # 不关系具体路径，只关心某个点是否被经过，且只经过一次，所以将n个点看做n个开关
F[1][0] = 0
def hamilton(n,weight,F):
    for i in range(1<<n):
        for j in range(n):
            if (i >> j &1):                         # 判断i状态里面是不是包含j
                for k in range(n):
                    if ((i^1<<j) >> k & 1):         # 把第j位去掉，且当前状态包含k
                        F[i][j] = min(F[i^1 << j][k]+weight[k][j])
    return F[(1<<n)-1][n-1]

2.2 起床困难综合征    
选择[0,m]之间的一个整数$x_0$，经给定的`n`次位运算，使结果`ans`最大。这题特点是**在二进制表示下不进位，可以单个位考虑**

* related blog
[起床困难综合症](https://blog.csdn.net/qq_39286580/article/details/107304377?utm_medium=distribute.pc_relevant.none-task-blog-2~default~baidujs_title~default-1.no_search_link&spm=1001.2101.3001.4242.2)

2.2.1 暴力搜索     
遍历[0,m]中每个数经过`n`次运算的结果，找出最大的值。O(m*n)，超时

In [ ]:
def suan(ops,nums,a):
    for op,num in zip(ops,nums):
        if op == 'AND':
            a &= num
        elif op == 'OR':
            a |= num
        elif op == 'XOR':
            a ^= num
    return a

def main(m,ops,nums):
    ans = 0
    for i in range(m):                          # 遍历[0,m]中每个数
        ans = max(ans,suan(i))
    print(ans)

2.2.2 bit贪心
从高位到低位按照bit位考虑（这样每个bit只有0或1），若$x_0$的第`k`位填1，应该满足下面两个条件:
1. 填1后，$x_0$不会超过`m`
2. 如果填1后,n次位运算结果为1


In [ ]:
def calc(ops,nums,bita,biti):
    for op,num in zip(ops,nums):
        x = num >> biti & 1             # 取出操作数的biti位，为了和bita计算
        if op == 'AND': 
            a &= x
        elif op =='OR':
            a |= x
        elif op == 'XOR':
            a ^= x
    return a

def main(m,ops,nums):
    val,ans = 0,0
    for biti in range(0,30)[::-1]:
        res0 = calc(ops,nums,0,biti)                # 当前位上0，得到的结果
        res1 = calc(ops,nums,1,biti)                # 当前位上1，得到的结果
        if (val + (1 << biti) <= m and res0 < res1):    # 满足上面的两个判断
            val += 1 << biti
            ans += res1 << biti
        else:
            ans += res0 << biti
    return val,ans

3. 成对变换     
对于**非负整数n**    
* n 是偶数, n xor 1 = n+1
* n 是奇数, n xor = n-1

因此，"0与1"，"2与3"，"4与5"构成成对变换

4. lowbit运算    
* 定义    
lowbit(n):非负整数`n`在二进制表示下“最低位的1及其后边所有的0”构成的数值。如`lowbit(10)=lowbit(1010)=2`

* 计算方式：
lowbit(n) = n & (~n+1) = n & -n
~n+1: 第k位=1，0~k-1位全为0，k+1~最高位，全为相反数

* 应用    
`lowbit`配合`Hash`得到**整数二进制表示下所有是1的位**
不断令n=n-lowbit(n)，直到n=0即可，每次得到得到的lowbit(n)使用$log_2 lowbit(n)$得到索引


In [ ]:
# 快速得到log2(lowbit(n))
## 第一种
H=[None] * (1<<20+1)
for i in range(21):
    H[1<<i] = i
## 第二种，利用数学技巧: 任取 k in [0,35], 2^k mod 37互不相等，且恰好取遍整数1~36
H = [None] * 37
for i in range(36):
     H[(1 << i) % 37] = i